In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf')

In [3]:
# Load the clothing image (e.g., a transparent PNG of a t-shirt)
clothing_img = cv2.imread("tshirt.png", -1)  # Ensure it's a transparent image (RGBA)

In [4]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

In [5]:
import cv2
import mediapipe as mp

def overlay_clothing(frame, clothing_img, landmarks):
    # Get coordinates of key body landmarks
    shoulder_left = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    shoulder_right = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    hip_left = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

    # Calculate width and height of the t-shirt based on the distance between shoulders and torso height
    width = int(abs(shoulder_right.x - shoulder_left.x) * frame.shape[1])
    height = int(abs(hip_left.y - shoulder_left.y) * frame.shape[0])

    # Ensure valid width and height
    if width <= 0 or height <= 0:
        return frame

    # Resize the t-shirt image to match calculated dimensions
    clothing_resized = cv2.resize(clothing_img, (width, height))

    # Calculate the center point between the shoulders
    center_x = int((shoulder_left.x + shoulder_right.x) / 2 * frame.shape[1])

    # Calculate top-left corner of the t-shirt for positioning
    top_left_x = center_x - width // 2
    top_left_y = int(shoulder_left.y * frame.shape[0])

    # Ensure the t-shirt stays within the frame boundaries
    if top_left_x + width > frame.shape[1]:
        width = frame.shape[1] - top_left_x
    if top_left_y + height > frame.shape[0]:
        height = frame.shape[0] - top_left_y

    # Resize the t-shirt again if necessary
    clothing_resized = cv2.resize(clothing_resized, (width, height))

    # Extract the alpha channel for transparency (assuming PNG with transparency)
    alpha_s = clothing_resized[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s

    # Overlay the t-shirt onto the frame
    for c in range(0, 3):  # Loop over color channels
        frame[top_left_y:top_left_y + height, top_left_x:top_left_x + width, c] = (
            alpha_s * clothing_resized[:, :, c] +
            alpha_l * frame[top_left_y:top_left_y + height, top_left_x:top_left_x + width, c]
        )

    return frame


In [6]:
"""def overlay_clothing(frame, clothing_img, landmarks):
    # Get the size of the clothing image
    clothing_h, clothing_w, _ = clothing_img.shape
    
    # Get body landmarks (shoulder and hip points)
    shoulder_left = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    shoulder_right = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    hip_left = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    hip_right = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

    # Calculate the region where the clothing should be placed
    width = int(abs(shoulder_right.x - shoulder_left.x) * frame.shape[1])
    height = int(abs(hip_left.y - shoulder_left.y) * frame.shape[0])

    # Ensure width and height are within bounds
    width = min(width, frame.shape[1])
    height = min(height, frame.shape[0])

    # Resize clothing image to fit on the person's body
    clothing_resized = cv2.resize(clothing_img, (width, height))

    # Find the coordinates for positioning the clothing on the body
    top_left_x = int(shoulder_left.x * frame.shape[1])
    top_left_y = int(shoulder_left.y * frame.shape[0])

    # Ensure that the placement is within the frame boundaries
    if top_left_x + width > frame.shape[1]:
        width = frame.shape[1] - top_left_x
    if top_left_y + height > frame.shape[0]:
        height = frame.shape[0] - top_left_y

    # Resize clothing image again to match the updated width and height if needed
    clothing_resized = cv2.resize(clothing_resized, (width, height))

    # Get the alpha channel of the clothing image for transparency
    alpha_s = clothing_resized[:, :, 3] / 255.0  # Alpha channel for clothing
    alpha_l = 1.0 - alpha_s  # Inverted alpha for background blending

    # Perform overlay only where dimensions match
    for c in range(0, 3):  # Loop over color channels
        frame[top_left_y:top_left_y + height, top_left_x:top_left_x + width, c] = (
            alpha_s * clothing_resized[:, :, c] +
            alpha_l * frame[top_left_y:top_left_y + height, top_left_x:top_left_x + width, c]
        )

    return frame
"""

"def overlay_clothing(frame, clothing_img, landmarks):\n    # Get the size of the clothing image\n    clothing_h, clothing_w, _ = clothing_img.shape\n    \n    # Get body landmarks (shoulder and hip points)\n    shoulder_left = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]\n    shoulder_right = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]\n    hip_left = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]\n    hip_right = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]\n\n    # Calculate the region where the clothing should be placed\n    width = int(abs(shoulder_right.x - shoulder_left.x) * frame.shape[1])\n    height = int(abs(hip_left.y - shoulder_left.y) * frame.shape[0])\n\n    # Ensure width and height are within bounds\n    width = min(width, frame.shape[1])\n    height = min(height, frame.shape[0])\n\n    # Resize clothing image to fit on the person's body\n    clothing_resized = cv2.resize(clothing_img, (width, height))\n\n    # Find the coordinates for positioning th

In [7]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break

    # Convert the frame to RGB as required by MediaPipe
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the image to detect the pose
    result = pose.process(frame_rgb)
    
    if result.pose_landmarks:
        mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Extract landmarks
        landmarks = result.pose_landmarks.landmark

        # Overlay clothing on detected landmarks
        frame = overlay_clothing(frame, clothing_img, landmarks)
    
    cv2.imshow('Virtual Dressing Room', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [4]:
#njnjnjnjnn